In [1]:
import jieba
from pyecharts.globals import SymbolType
from pyecharts.charts import Bar,WordCloud
from pyecharts import options as opts
import jieba.posseg as pseg
import pandas as pd
import re
from collections import Counter
import os

In [2]:
df = pd.read_csv(r'C:\Users\28934\Desktop\comment_data.txt')
comment = df['comment'].values.tolist()
#将评论内容转换为列表

In [3]:
word_dict = {}
for i in comment:
    words = jieba.cut(i)
    #生成一个生成器
    for j in words:
    #遍历words列表
        if j not in word_dict:
            word_dict[j] = 1
        #如果该词汇(索引)尚未加入到字典中，则设置其值(出现次数)为1
        else:
            word_dict[j] += 1
        #如果该词汇(索引)已出现在字典中，则设置其值(出现次数)加1

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\28934\AppData\Local\Temp\jieba.cache
Loading model cost 0.652 seconds.
Prefix dict has been built successfully.


In [4]:
word_dict

{'依旧': 50,
 '跨': 2,
 '时代': 75,
 '，': 5699,
 'Jay': 47,
 '没有': 326,
 '过去': 39,
 '为啥': 14,
 '取景': 1,
 '要': 190,
 '在': 607,
 '东京': 1,
 ' ': 3225,
 '国内': 1,
 '找': 31,
 '不到': 30,
 '更好': 19,
 '的': 5713,
 '？': 580,
 '杰哥': 4,
 '好听': 297,
 '歌词': 92,
 '写得': 1,
 '真好': 2,
 '最': 256,
 '喜欢': 513,
 '歌手': 79,
 '从': 131,
 '初': 4,
 '2': 17,
 '那年': 14,
 '龙卷风': 36,
 '开始': 83,
 '。': 1729,
 '不管怎么': 2,
 '说': 399,
 '其实': 35,
 '我': 3475,
 '还': 283,
 '爱': 349,
 '你': 2889,
 '96': 1,
 '年': 141,
 '社畜': 2,
 '为了': 260,
 '周杰伦': 1136,
 '开': 33,
 '绿': 119,
 '钻': 53,
 '觉得': 119,
 '太': 76,
 '值得': 24,
 '了': 2344,
 '这': 319,
 '是': 2077,
 '成长': 19,
 '旋律': 57,
 '啊': 286,
 '😭': 261,
 '很': 420,
 '久': 25,
 '今天': 62,
 '第一次': 110,
 '真正': 11,
 '意义': 10,
 '上': 158,
 '买': 210,
 '他': 640,
 '歌': 469,
 '听': 847,
 '整首歌': 4,
 '下来': 10,
 '心里': 41,
 '起': 12,
 '涟漪': 2,
 '只有': 84,
 '一句': 25,
 '：': 173,
 '什么': 358,
 '都': 696,
 '支持': 169,
 '梦想': 55,
 '让': 299,
 '有': 786,
 '波动': 1,
 '！': 1270,
 '周董': 265,
 '回不来': 2,
 '吗': 285,
 '评论': 78,
 '一万八'

In [5]:
stop = pd.read_csv(r'C:\Users\28934\Desktop\Chinese_Stopwords.txt',header = None)
#此处设置第一行即表头为None，避免将第一个字符设置为表头
stop.columns = ['word']
stop_1 = [' '] + list(stop.word)
#由于该文件下没有' '即空格符，所以在此处加上

In [6]:
for i in range(len(stop_1)):
    if stop_1[i] in word_dict:
        word_dict.pop(stop_1[i])
    #如果字典的索引中存在stop_1中的元素，则删除，这步的目的是过滤掉无用的词汇

In [7]:
word_dict = sorted(word_dict.items(),key = lambda x:x[1],reverse = True)
#以字典的值为依据进行排序处理

In [8]:
words = word_dict[:101]
#将出现频率top100作为列表放入变量words中

In [9]:
wordcloud = WordCloud()
wordcloud.add('',words,word_size_range = [20,100],shape = 'circle')
wordcloud.render_notebook()

In [10]:
def speech_cut(speech):
    word_list = []
    for word in word_dict:
        words = pseg.cut(word[0])
        for w, flag in words:
            if flag == speech:
                word_list.append(word)
    return word_list
#pseg是提取词性的方法

In [11]:
verb_word = speech_cut('v')
wordcloud_1 = WordCloud()
wordcloud_1.add("", verb_word[:101], word_size_range=[20, 100], shape='circle')
wordcloud_1.render_notebook()
#v代表verb即动词，这次词云统计将统计所有动词

In [12]:
non_word = speech_cut('n')
wordcloud_2 = WordCloud()
wordcloud_2.add("", non_word[:101], word_size_range=[20, 100], shape='circle')
wordcloud_2.render_notebook()
#同理，此处提取名词

In [13]:
def get_emoji(content):
    pattern = re.compile(u'[\U00010000-\U0010ffff]')
    #定义一个正则表达式，表示匹配所有表情符号
    result = re.findall(pattern,content)
    #提取出值中所有的表情符号成列表，若无表情则提取为空列表
    return result

In [14]:
df['emoji'] = df['comment'].apply(get_emoji)
#在原文本中添加一列emoji以记录

In [15]:
emoji_list = df['emoji'].values.tolist()
emoji = sum(emoji_list,[])
#将二维列表降维，形成一个由单个emoji为元素的一维列表

In [16]:
emoji_dict = Counter(emoji)
#利用Counter方法直接计算列表中每个元素出现次数，并以元素为索引，次数为值输出一个字典

In [19]:
y_emojis, x_counts = zip(*emoji_dict.most_common())
#*zip方法可以将字典的键和值分别提取成两个元组
#most_common方法可以提取出值最高的前几位数据
bar = Bar()
bar.add_xaxis(y_emojis[:20])
bar.add_yaxis('',x_counts[:20])
#分别取前19个数据进行统计
#纵坐标必须添加标签名，此处不添加即为''
bar.set_global_opts(title_opts=opts.TitleOpts(title="emoji频率统计", subtitle=" "))
bar.render_notebook()